QLIP
====

**QLIP: Text-Aligned Visual Tokenization Unifies Auto-Regressive Multimodal Understanding and Generation**

* Paper: https://arxiv.org/pdf/2502.05178

![QLIP Overview](../assets/qlip_overview.png)


```bash
git clone https://github.com/NVlabs/QLIP.git

pip install torch torchvision
pip install transformers
```

### Load model

In [1]:
import sys
import torch
from transformers import CLIPImageProcessor, CLIPTokenizer
from torchvision.transforms import Normalize
from PIL import Image

sys.path.append("QLIP/QLIP")  # must clone repo
from modeling_qlip import QLIPModel

processor = CLIPImageProcessor.from_pretrained(
    "nvidia/QLIP-B-8-256"
)
tokenizer = CLIPTokenizer.from_pretrained(
    "nvidia/QLIP-B-8-256"
)
model = QLIPModel.from_pretrained(
    "nvidia/QLIP-B-8-256"
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.eval().to(device);

/home/pyml/anaconda3/envs/py311/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of QLIPModel were not initialized from the model checkpoint at nvidia/QLIP-B-8-256 and are newly initialized: ['vision_decoder.encoder.layers.0.self_attn.rope.freqs_cos', 'vision_decoder.encoder.layers.0.self_attn.rope.freqs_sin', 'vision_decoder.encoder.layers.1.self_attn.rope.freqs_cos', 'vision_decoder.encoder.layers.1.self_attn.rope.freqs_sin', 'vision_decoder.encoder.layers.10.self_attn.rope.freqs_cos', 'vision_decoder.encoder.layers.10.self_attn.rope.freqs_sin', 'vision_decoder.encoder.layers.11.self_attn.rope.freqs_cos', 'vision_decoder.encoder.layers.11.self_attn.rope.freqs_sin', 'vision_decoder.encoder.layers.2.self_attn.rope.freqs_cos', 'vision_decoder.encoder.layers.2.self_attn.rope.freqs_sin', 'visio